<a href="https://colab.research.google.com/github/IEEE-NITK/Quantum_computing/blob/main/Learning/Sujay/Adjoint_differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Created on 1st February 2022 20:39

@author:Sujay Chuttar

Performing adjoint differentiation in quantum circuits
"""

In [4]:
import pennylane as qml
from pennylane import numpy as np

In [6]:
#device to simulate algorithm on
dev = qml.device('default.qubit', wires = 2)

x = ([0.1, 0.2, 0.3])

@qml.qnode(dev, diff_method = 'adjoint')
def circuit(a):
  qml.RX(a[0], wires = 0)
  qml.CNOT(wires = [0, 1])
  qml.RY(a[1], wires = 1)
  qml.RZ(a[2], wires = 1)
  return qml.expval(qml.PauliX(wires = 1))

In [7]:
n_gates = 4
n_params = 3

ops = [
       qml.RX(x[0], wires = 0),
       qml.CNOT(wires = (0, 1)),
       qml.RY(x[1], wires = 1),
       qml.RZ(x[2], wires = 1)
]
M = qml.PauliX(wires = 1)

In [8]:
state = dev._create_basis_state(0)

for op in ops:
  state = dev._apply_operation(state, op)

print(state)

[[9.82601808e-01-0.14850574j 9.85890302e-02+0.01490027j]
 [7.45635195e-04+0.00493356j 7.43148086e-03-0.04917107j]]


In [9]:
bra = dev._apply_operation(state, M)
ket = state

In [10]:
M_expval = np.vdot(bra, ket)
print("vdot: ", M_expval)
print("Qnode: ", circuit(x))

vdot:  (0.18884787122715618+3.634721684493463e-19j)
Qnode:  0.18884787122715618


In [11]:
bra_n = dev._create_basis_state(0)

for op in ops:
  bra_n = dev._apply_operation(bra_n, op)

bra_n = dev._apply_operation(bra_n, M)
bra_n = dev._apply_operation(bra_n, ops[-1].inv())

ops[-1].inv() #returning the operation to an uninverted state

ket_n = dev._create_basis_state(0)

for op in ops[:-1]:
  ket_n = dev._apply_operation(ket_n, op)

M_expval_n = np.vdot(bra_n, ket_n)
print(M_expval_n)

(0.18884787122715616+1.9739809094676298e-18j)


In [12]:
#version 2 method
bra_n_v2 = dev._apply_operation(state, M)
ket_n_v2 = state

ops[-1].inv()

bra_n_v2 = dev._apply_operation(bra_n_v2, ops[-1])
ket_n_v2 = dev._apply_operation(ket_n_v2, ops[-1])

M_expval_n_v2 = np.vdot(bra_n_v2, ket_n_v2)
print(M_expval_n_v2)

(0.18884787122715613+2.9931365520227565e-18j)


In [13]:
#derivative of an operator
grad_op0 = qml.operation.operation_derivative(ops[0])
print(grad_op0)

[[-0.02498958+0.j          0.        -0.49937513j]
 [ 0.        -0.49937513j -0.02498958+0.j        ]]


In [14]:
bra = dev._apply_operation(state, M)
ket = state

grads = []

for op in reversed(ops):
  op.inv()
  ket = dev._apply_operation(ket, op)

  #Calculating derivative
  if op.num_params != 0:
    dU = qml.operation.operation_derivative(op)

    bra_temp = dev._apply_unitary(bra, dU, op.wires)

    dM = 2 * np.real(np.vdot(bra_temp, ket))
    grads.append(dM)

  bra = dev._apply_operation(bra, op)
  op.inv()

#Reverse the order of gradients
#Since we calculated them in reverse order
grads = grads[::-1]

print("Our calculation : ", grads)

grad_compare = qml.grad(circuit)(x)
print("comparison : ", grad_compare)

Our calculation :  [-0.018947989233612104, 0.9316157966884513, 0.05841749223216956]
comparison :  [array(-0.01894799), array(0.9316158), array(-0.05841749)]


/usr/local/lib/python3.7/dist-packages/pennylane/_grad.py:100: UserWarning: Starting with PennyLane v0.21.0, when using Autograd, inputs have to explicitly specify requires_grad=True (or the argnum argument must be passed) in order for trainable parameters to be identified.
  UserWarning,


In [15]:
#Lightning simulator
dev_lightning = qml.device('lightning.qubit', wires = 2)

@qml.qnode(dev_lightning, diff_method = "adjoint")
def circuit_adjoint(a):
  qml.RX(a[0], wires = 0)
  qml.CNOT(wires = (0, 1))
  qml.RY(a[1], wires = 1)
  qml.RZ(a[2], wires = 1)
  return qml.expval(M)

qml.grad(circuit_adjoint)(x)

/usr/local/lib/python3.7/dist-packages/pennylane/_grad.py:100: UserWarning: Starting with PennyLane v0.21.0, when using Autograd, inputs have to explicitly specify requires_grad=True (or the argnum argument must be passed) in order for trainable parameters to be identified.
  UserWarning,


[array(-0.01894799), array(0.9316158), array(-0.05841749)]